<font size="5">__multi-class classification model for product category prediction: Data Manipulation__

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from warnings import simplefilter

%matplotlib inline

<font size="3">I decided to use the customers activities in the category he has purchased from __only in the same date as the order date__.
<br>activities who didn't end up in a purchase will recieve category 0. this will empower the model by abling it to also predict if a costumer will not purchase at all.<br>
activities that were made at the same day but after the order will be deleted from the data, as those rows represent activities can't influence the order since it already happen.

In [2]:
# importing the data:
online = pd.read_csv("online.csv")
order = pd.read_csv("order.csv")

joining the order data to the online data according based on the category,date and custno:

In [3]:
#creating string column for date string only:
remove_time_string= lambda x : x[:10]
online["date_string"]=online["dt"].apply(remove_time_string)
order["date_string"]=order["orderdate"].apply(remove_time_string)

#turning custno prodcat1, and category to string :
online['custno'] = online['custno'].apply(str)
online['category'] = online['category'].apply(str)

order['custno'] = order['custno'].apply(str)
order['prodcat1'] = order['prodcat1'].apply(str)

# concating category(prodcat1)+custno+date_string:
online["cat+custno+date"] = online["category"] +'_'+ online["custno"] +'_'+ online["date_string"]
order["cat+custno+date"] = order["prodcat1"] +'_'+ order["custno"] +'_'+ order["date_string"]


In [4]:
#removing duplilcate columns from order to enable joining with online:
order_for_join=order.drop(columns=['custno', 'date_string'])

#joining the two tables while keeping online lines that don't correspond (left join):
joined_data=online.set_index('cat+custno+date').join(order_for_join.set_index('cat+custno+date'),how='left')
joined_data.sort_values(by='revenue', ascending=False,inplace=True)

Removing activities that occurred after the purchase:

In [5]:
joined_data.fillna(0,inplace=True)

# turning dt and orderdate columns to datetime objects:
joined_data['dt']= pd.to_datetime(joined_data['dt'])
joined_data['orderdate']= pd.to_datetime(joined_data['orderdate'])

#checking which rows represent activities that happen after the purchase:
joined_data['activity_after_order'] = np.where((joined_data['dt']>=joined_data['orderdate']) & (joined_data['ordno']!=0),True,False)

#removing rows that represent activities that happen after the purchase:
joined_data=joined_data[joined_data['activity_after_order']==False]


<font size="3">The next steps will involve transforming the data to having a unique row per each order and adding the variables I suspect to influence the target variable (the order category): <br>
    
- __event1 and event2 values:__ assuming those values are ids and both are created due to the same activity, the best practice is to combined them as one. the way I think will generate the most learning from them is to turn each combined value to a column as its value will be the number of times it occurred per "order"

- __number of activities:__ total numbers of activities (event1 and 2 combination) each order generated.

- __seconds diff between first and last activity:__ the time which passed between the first activity and the last one per "order".

Removing prodcat2 and revenue columns (unnecessary for the model and create 'duplicate lines'):

In [6]:
joined_data.drop(['revenue', 'prodcat2'], axis=1,inplace=True)
joined_data.drop_duplicates(inplace=True)

adding a column that represent the events combinations (event1+event2)

In [7]:
# concating event2 & event1:
joined_data['event1'] = joined_data['event1'].apply(int)
joined_data['event2'] = joined_data['event2'].apply(int)

joined_data['event1'] = joined_data['event1'].apply(str)
joined_data['event2'] = joined_data['event2'].apply(str)

joined_data['event1_event2_combined']=joined_data['event1'] +'_'+ joined_data['event2']

creating a unique row per order table and adding extra variables ('seconds diff between first and last activity', 'num of activities'):

In [8]:
#grouping the data and aggregating wanted variables:
joined_data_raw_per_order=joined_data.groupby(['cat+custno+date','custno','category','ordno','orderdate','prodcat1']).agg({'dt': ['min', 'max'],'event1_event2_combined':['count']})

#changing the new columns names and removing the indexing:
joined_data_raw_per_order.columns = ['dt_min', 'dt_max','activities_per_order']
joined_data_raw_per_order.reset_index(inplace=True)

# generating the seconds diff column:
joined_data_raw_per_order['total_activities_time_seconds']=(joined_data_raw_per_order.dt_max-joined_data_raw_per_order.dt_min).dt.seconds

turning each combined value of the events to a column and counting their occurances per "order":

In [9]:
# trying to apply pivot func:
joined_data_grouped_for_pivot=joined_data.groupby(['cat+custno+date','ordno','orderdate','prodcat1','event1_event2_combined'])['event1_event2_combined'].count()
joined_data_grouped_for_pivot=joined_data_grouped_for_pivot.rename(columns={'event1_event2_combined':'event1_event2_combined_count'})
joined_data_grouped_for_pivot=joined_data_grouped_for_pivot.to_frame()
joined_data_grouped_for_pivot.reset_index(inplace=True)
joined_data_grouped_for_pivot=joined_data_grouped_for_pivot.rename(columns={0:'event1_event2_combined_count'})

joined_data_grouped_for_pivot['cat+custno+date,ordno,orderdate,prodcat1'] = joined_data_grouped_for_pivot['cat+custno+date'].astype(str) +'_'+ joined_data_grouped_for_pivot['ordno'].astype(str) +'_'+ joined_data_grouped_for_pivot['orderdate'].astype(str)+'_'+ joined_data_grouped_for_pivot['prodcat1'].astype(str)
joined_data_grouped_for_pivot.set_index('cat+custno+date,ordno,orderdate,prodcat1',inplace=True)

joined_data_grouped_for_pivot.reset_index(inplace=True)
activities_count_df=pd.pivot_table(joined_data_grouped_for_pivot, values='event1_event2_combined_count', index='cat+custno+date,ordno,orderdate,prodcat1',columns=['event1_event2_combined'], aggfunc=np.sum)
                                                       
activities_count_df.reset_index(inplace=True)

In [10]:
#preparing "joined_data_raw_per_order" the data for merging with "activities_count_df" by creating a common column:
joined_data_raw_per_order['cat+custno+date,ordno,orderdate,prodcat1'] = joined_data_raw_per_order['cat+custno+date'].astype(str) +'_'+ joined_data_raw_per_order['ordno'].astype(str) +'_'+ joined_data_raw_per_order['orderdate'].astype(str)+'_'+ joined_data_raw_per_order['prodcat1'].astype(str)

#merging the tables:
joined_data=joined_data_raw_per_order.merge(activities_count_df, left_on=['cat+custno+date,ordno,orderdate,prodcat1'], right_on=['cat+custno+date,ordno,orderdate,prodcat1'])
joined_data.reset_index(inplace=True)

# extracting a file for the model deployment ('Machine Learning.ipynb'):
joined_data.to_csv('final_manipulated_data_ready_for_models.csv')

The Machine Learning part is in the Jupyter notebook file: "'Machine Learning.ipynb'"